In [9]:
from urllib.request import Request, urlopen, urlretrieve
from urllib.parse import urljoin
import requests
import re
import os
import pandas as pd
import numpy as np
import copy
from datetime import datetime
from datetime import timedelta
from selenium.webdriver import Firefox
from selenium.webdriver.firefox.options import Options

rootdir = "C:/Users/minif/OneDrive/Desktop/"
data = pd.read_excel(rootdir + "Biotech_Companies_Neural_Net_Data.xlsx")
expiration_data = pd.read_excel(rootdir + "Options_Expiration_Dates.xlsx")

start_dates_unix = data['UNIX Timestamp Start'].values.tolist()
end_dates_unix = data['UNIX Timestamp End'].values.tolist()
tickers = data['Tickers'].values.tolist()
expiration_dates = expiration_data['Unix Timestamp'].values.tolist()

start_dates_unix = list(map(int, start_dates_unix))
end_dates_unix = list(map(int, end_dates_unix))
expiration_dates = list(map(int, expiration_dates))


'''
So, basically, I need to get the strike prices for every single date that I'm trying to find.
Only problem is there is no database (free) of just strike prices.
But, I can find the strike prices from quantconnect by looking at the tree and then determining the difference in
strike prices because the difference between strike prices is always constant.
So, after I find the difference, I know the lowest strike price, because that is the first one shown on the page.
The last one is going to have to be itteravely run to be found, going through calling each HTML page
until one of them does not work, and then we know that the previous one was the highest value that you have.
Then, we have the strike price range, and can pull the data for the option chain for a specific date,
for specific expiration dates, for a specific ticker, and given a specific strike price to look at.
This is going to be a dusy, but it's possible and that is the good part.
I am going to use this script to pull the pictures of all the data and then I will write a script to analyze
all the pictures to find the text.
And finally, after all of this, we will have access to a free database of historical, minute by minute,
option chains for over 4000 stocks being traded in the US options market.  God help us.  We are not downloading
their entire database.  Just what we need.  If we wanted to get everything, though, we could use the computers
at the Matten Center and then have maybe two or three running with the scripts on them as they pull all 4000
for all possible dates.

UPDATE: Getting the data has proven to be more difficult because quantconnect did something clever: whenever you call
the get
'''
data

,Tickers,Market Start Date,Market End Date,UNIX Timestamp Start,UNIX Timestamp End,Open Start,High Start,Low Start,Close Start,Open End,...,Close End,Call Price Before,Put Price Before,Call Price After,Put Price After,Call Net,Put Net,Profit,Experiation Date,Unix Timestamp Expirations
0,SPPI,2009-07-06,2009-07-09,1246838400,1247097600,5.78,5.96,5.30,5.40,5.30,...,5.23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-01-16,1.232078e+09
1,ARDM,2009-07-16,2009-07-21,1247702400,1248134400,8.40,12.80,8.40,11.20,8.00,...,8.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-02-20,1.235102e+09
2,SPPI,2009-09-04,2009-09-08,1252022400,1252368000,8.94,10.00,7.90,8.33,8.19,...,7.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-03-20,1.237522e+09
3,SPPI,2009-10-09,2009-10-13,1255046400,1255392000,4.98,5.44,4.82,5.09,4.86,...,4.56,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-04-17,1.239941e+09
4,GTXI,2009-11-02,2009-11-05,1257120000,1257379200,58.00,59.00,44.90,45.60,37.20,...,38.10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-05-15,1.242360e+09
5,PRTK,2009-11-02,2009-11-05,1257120000,1257379200,63.00,66.00,60.96,64.80,55.32,...,64.80,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-06-19,1.245384e+09
6,HEB,2009-12-01,2009-12-04,1259625600,1259884800,14.64,14.88,14.04,14.40,8.04,...,7.44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-07-17,1.247803e+09
7,FCSC,2009-12-18,2009-12-22,1261094400,1261440000,457.50,457.50,450.00,450.00,393.75,...,427.50,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-08-14,1.250222e+09
8,ACOR,2010-01-22,2010-01-26,1264118400,1264464000,25.71,29.36,25.10,28.12,27.79,...,27.48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-09-18,1.253246e+09
9,ALKS,2010-01-27,2010-02-02,1264550400,1265068800,11.55,11.68,11.40,11.50,11.08,...,11.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2009-10-16,1.255666e+09


In [2]:
def pull_html_option(ticker, date, price, expiration_date, call_put):
    # date and expiration_date should be in form of YYYYMMDD
    # ticker should be lower case
    # price should be the strike price times 10,000
    
    try:
        r = requests.get("https://www.quantconnect.com/data/file/option/usa/minute/" + ticker + "/" + date + "_quote_american.zip/" + date + "_sppi_minute_quote_american_" + call_put + "_" + price + "_" + expiration_date + ".csv")
        
    except Exception :
        print("An internet error has occured.")
        return

    if r.status_code == 200:
        r = Request("https://www.quantconnect.com/data/file/option/usa/minute/" + ticker + "/" + date + "_quote_american.zip/" + date + "_sppi_minute_quote_american_" + call_put + "_" + price + "_" + expiration_date + ".csv")
        print("https://www.quantconnect.com/data/file/option/usa/minute/" + ticker + "/" + date + "_quote_american.zip/" + date + "_sppi_minute_quote_american_" + call_put + "_" + price + "_" + expiration_date + ".csv")
        u = urlopen(r)
        html = u.read().decode('utf-8')
        return html
    else:
        print("An HTTP Error has Occured with pull_html(" + ticker + " " + date + ") Error Code: "+str(r.status_code))

In [3]:
def pull_html_all_options(ticker, date):
    try:
        r = requests.get("https://www.quantconnect.com/data/tree/option/usa/minute/" + ticker + "/" + date + "_quote_american.zip")
        
    except Exception :
        print("An internet error has occured.")
        return

    if r.status_code == 200:
        r = Request("https://www.quantconnect.com/data/tree/option/usa/minute/" + ticker + "/" + date + "_quote_american.zip")
        print("https://www.quantconnect.com/data/tree/option/usa/minute/" + ticker + "/" + date + "_quote_american.zip")
        u = urlopen(r)
        html = u.read().decode('utf-8')
        return html
    else:
        print("An HTTP Error has Occured with pull_html(" + ticker + " " + date + ") Error Code: "+str(r.status_code))

In [4]:
def test_html(url):
    try:
        r = requests.get(url)
        
    except Exception:
        print("An internet error has occured.")
        return
    
    if r.status_code == 200:
        r = Request(url)
        print(url)
        u = urlopen(r)
        html = u.read().decode('utf-8')
        return html
    else:
        print("An HTTP error has occured.")

In [5]:
def convert_date_quantconnect(date):
    #YYYYMMDD
    return datetime.utcfromtimestamp(date).strftime('%Y%m%d')

In [6]:
def determine_expiration_dates(date):
    #finds the next three expiration dates after the date
    temp_dates = copy.deepcopy(expiration_dates)
    #print(temp_dates)
    temp_dates.append(date)
    sorted_dates = sorted(temp_dates, key=float)
    
    index = sorted_dates.index(date)
    if index == len(sorted_dates) - 1:
        return None
    elif index == len(sorted_dates) - 2:
        return [sorted_dates[index + 1]]
    elif index == len(sorted_dates) - 3:
        return [sorted_dates[index + 1], sorted_dates[index + 2]]
    elif index <= len(sorted_dates) -3:
        return [sorted_dates[index + 1], sorted_dates[index + 2], sorted_dates[index + 3]]

In [7]:
#def find_delta_strike(html):
    

In [8]:
#print(convert_date_quantconnect(determine_expiration_dates(1308873600)[0]))
#pull_html_all_options('drrx',convert_date_quantconnect(1308873600))
test_html("https://www.quantconnect.com/data/file/option/usa/minute/drrx/20110624_quote_american.zip/20110624_drrx_minute_quote_american_call_50000_20110820.csv")

https://www.quantconnect.com/data/file/option/usa/minute/drrx/20110624_quote_american.zip/20110624_drrx_minute_quote_american_call_50000_20110820.csv


'<!DOCTYPE html>\n<html xmlns="http://www.w3.org/1999/xhtml" id="QuantConnect" class="quantconnect" prefix="og: http://ogp.me/ns#">\n<head>\n    <meta charset="UTF-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1">\n    <meta property="og:title" content="QuantConnect Data Explorer: DRRX - DURECT CORP"> \n<meta property="og:image" content="https://www.quantconnect.com/DataExplorerMetaImage/drrx.png"> \n<meta name="description" content="DRRX is provided by AlgoSeek and is available since January 1st, 2010, in the resolutions  and Minute. QuantConnect provides a free algorithm backtesting tool and financial data so engineers can design algorithmic trading strategies."> \n<meta property="og:url" content="https://www.quantconnect.com/data/file/option/usa/minute/drrx/20110624_quote_american.zip/20110624_drrx_minute_quote_american_call_50000_20110820.csv"> \n<meta property="og:type" content="Company"> \n<meta property="og:site_name" content="Free Data Available | Ex